In [26]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [27]:
llm = OllamaFunctions(model = 'llama3',
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]),
             format='json')

In [28]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


# Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")


# Prompt template
prompt = PromptTemplate.from_template(
    """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde.

Human: {question}
AI: """
)

# Chain
structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

In [29]:
alex = chain.invoke("Describe Alex")
alex

{
"tool": "Person",
"tool_input": {
"name": "Alex",
"height": 5,
"hair_color": "blonde"
}
}

Person(name='Alex', height=5.0, hair_color='blonde')

In [30]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class output(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [31]:
query = 'Tell me a joke'

parser = JsonOutputParser(pydantic_object=output)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": query})

{"tool": "__conversational_response", "tool_input": {"response": "Why did the scarecrow win an award? Because he was outstanding in his field!"}}

OutputParserException: Invalid json output: Why did the scarecrow win an award? Because he was outstanding in his field!

SyntaxError: invalid syntax (3559697825.py, line 1)